In [1]:
import palimpzest as pz
import pandas as pd
import time
import os
import IPython

formatter = IPython.get_ipython().display_formatter.formatters['text/plain']
formatter.max_seq_length = 0

# set OPENAI_API_KEY environment variable based on OPENAI_API_KEY
# os.environ["OPENAI_API_KEY"] = "{{ OPENAI_API_KEY }}" 

sci_paper_cols = [
    {"name": "title", "type": str, "desc": "The title of the paper. This is a natural language title, not a number or letter."},
    {"name": "author", "type": str, "desc": "The name of the first author of the paper"},
    {"name": "abstract", "type": str, "desc": "A short description of the paper contributions and findings"},
]

reference_cols = [
    {"name": "index", "type": int, "desc": "The index of the reference in the paper"},
    {"name": "title", "type": str, "desc": "The title of the paper being cited"},
    {"name": "first_author", "type": str, "desc": "The author of the paper being cited"},
    {"name": "year", "type": int, "desc": "The year in which the cited paper was published"},
]

print("Setup complete")

Setup complete


What are the datasets available in the system?

In [2]:
import pandas as pd
from prettytable import PrettyTable

dataset_path = "/home/gerardo/bdf-pz/testdata"
ds = os.listdir(dataset_path)

# construct table for printing
records = [["Name", "Path", "N. Files"]]
for path in ds:
    abspath = os.path.join(dataset_path,path)
    records.append([path,abspath,len(os.listdir(abspath))])

# print table of registered datasets
t = PrettyTable(records[0])
t.add_rows(records[1:])
print(t)

+-------------------+-------------------------------------------------+----------+
|        Name       |                       Path                      | N. Files |
+-------------------+-------------------------------------------------+----------+
|  sigmod-tiny-txt  |  /home/gerardo/bdf-pz/testdata/sigmod-tiny-txt  |    2     |
| bdf-usecase3-tiny | /home/gerardo/bdf-pz/testdata/bdf-usecase3-tiny |    4     |
|    sigmod-tiny    |    /home/gerardo/bdf-pz/testdata/sigmod-tiny    |    2     |
|    sigmod-demo    |    /home/gerardo/bdf-pz/testdata/sigmod-demo    |    11    |
|      bdf-demo     |      /home/gerardo/bdf-pz/testdata/bdf-demo     |    11    |
+-------------------+-------------------------------------------------+----------+


Show me the files available in bdf-demo

In [4]:
import os
files = os.listdir("/home/gerardo/bdf-pz/testdata/bdf-demo")

files

['bolderson2010.pdf',
 'bonfiglio2023.pdf',
 'marques2021.pdf',
 'brunyanszki2014.pdf',
 'liu2024.pdf',
 'li2021.pdf',
 'zethoven2022.pdf',
 'vasaikar2019.pdf',
 'ceccaldi2015.pdf',
 'munkhbaatar2020.pdf',
 'gupte2017.pdf']

Create a schema to extract authors.

Consider the bdf-demo dataset. This is a dataset of scientific papers. Extract all authors of papers about brain cancer.

In [1]:
import palimpzest as pz
dataset = pz.Dataset("testdata/sigmod-tiny")

# Define a class name
author_columns = [{'name': 'author_name', 'type': str, 'desc': 'The full name of the author.'},
 {'name': 'author_affiliation',
  'type': str,
  'desc': 'The affiliation of the author.'},
 {'name': 'author_email', 'type': str, 'desc': 'The email address of the author.'}]


cardinality = pz.Cardinality.ONE_TO_MANY
dataset = dataset.sem_add_columns(author_columns, cardinality=cardinality)

policy = pz.MaxQuality()

config = pz.QueryProcessorConfig(
    policy=policy,
    cache=False,
    verbose=False,
    execution_strategy="parallel",
    allow_code_synth=False,
)

results = dataset.run(config)
results_df = results.to_df()

print(list(results.execution_stats.plan_strs.values())[0])
display(results_df)


Output()

PROMPT:
You are a helpful assistant whose job is to generate a JSON object.
You will be presented with a context and a set of output fields to generate. Your task is to generate a JSON object
which fills in the output fields with the correct values.
You will be provided with a description of each input field and each output field. All of the fields in the output 
JSON object can be derived using information from the context.

Remember, your answer must be a valid JSON list of dictionaries. The list may contain one or more dictionaries, and
each dictionary should only have the specified output fields. Finish your response with a newline character 
followed by ---
---
INPUT FIELDS:
- contents: The contents of the file
- filename: The UNIX-style name of the file
- text_contents: The text-only contents of the PDF

OUTPUT FIELDS:
- author_affiliation: The affiliation of the author.
- author_email: The email address of the author.
- author_name: The full name of the author.

CONTEXT:
{
  "contents": "<bytes>",
  "filename": "vasaikar2019.pdf",
  "text_contents": "Resource\nProteogenomic Analysis of Human Colon Cancer\nReveals New Therapeutic 
Opportunities\nSuhas Vasaikar,1,2,14 Chen Huang,1,2,14 Xiaojing Wang,1,2,12,14 Vladislav A. Petyuk,3,14 Sara R. 
Savage,4,14 Bo Wen,1,2\nYongchao Dou,1,2 Yun Zhang,1 Zhiao Shi,1,2 Osama A. Arshad,3 Marina A. Gritsenko,3 Lisa J. 
Zimmerman,5\nJason E. McDermott,3 Therese R. Clauss,3 Ronald J. Moore,3 Rui Zhao,3 Matthew E. Monroe,3 Yi-Ting 
Wang,3\nMatthew C. Chambers,5 Robbert J.C. Slebos,5 Ken S. Lau,6 Qianxing Mo,7,13 Li Ding,8 Matthew 
Ellis,1,7\nMathangi Thiagarajan,9 Christopher R. Kinsinger,10 Henry Rodriguez,10 Richard D. Smith,3 Karin D. 
Rodland,3,11,15,*\nDaniel C. Liebler,5,15,* Tao Liu,3,15,* Bing Zhang,1,2,7,15,16,* and Clinical Proteomic Tumor 
Analysis Consortium\n1Lester and Sue Smith Breast Center, Baylor College of Medicine, Houston, TX 77030, 
USA\n2Department of Molecular and Human Genetics, Baylor College of Medicine, Houston, TX 77030, USA\n3Biological 
Sciences Division, Paci\ufb01c Northwest National Laboratory, Richland, WA 99352, USA\n4Department of Biomedical 
Informatics, Vanderbilt University Medical Center, Nashville, TN 37232, USA\n5Department of Biochemistry, 
Vanderbilt University, Nashville, TN 37232, USA\n6Department of Cell and Developmental Biology, Vanderbilt 
University, Nashville, TN 37232, USA\n7Dan L Duncan Comprehensive Cancer Center, Baylor College of Medicine, 
Houston, TX 77030, USA\n8The McDonnell Genome Institute, Washington University in St. Louis, Forest Park Avenue, 
Campus Box 8501, St. Louis, MO 63108, USA\n9Leidos Biomedical Research Inc., Frederick National Laboratory for 
Cancer Research, Frederick, MD 21702, USA\n10Of\ufb01ce of Cancer Clinical Proteomics Research, National Cancer 
Institute, Bethesda, MD 20892, USA\n11Department of Cell, Developmental, and Cancer Biology, Oregon Health & 
Science University, Portland, OR 97221, USA\n12Present address: Department of Epidemiology and Biostatistics, 
Greehey Children\u2019s Cancer Research Institute, The University of Texas\nHealth Science Center at San Antonio, 
San Antonio, TX 78229, USA\n13Present address: Department of Biostatistics & Bioinformatics, H. Lee Mof\ufb01tt 
Cancer Center & Research Institute, Tampa, FL 33612, USA\n14These authors contributed equally\n15Senior 
author\n16Lead Contact\n*Correspondence: karin.rodland@pnnl.gov(K.D.R.), daniel.liebler@vanderbilt.edu(D.C.L.), 
tao.liu@pnnl.gov(T.L.), bing.zhang@bcm.edu(B.Z.)\nhttps://doi.org/10.1016/j.cell.2019.03.030\nSUMMARY\nWe performed
the \ufb01rst proteogenomic study on\na prospectively collected colon cancer cohort.\nComparative proteomic and 
phosphoproteomic\nanalysis of paired tumor and normal adjacent tis-\nsues produced a catalog of colon 
cancer-associ-\nated proteins and phosphosites, including known\nand putative new biomarkers, drug targets, 
and\ncancer/testis antigens. Proteogenomic integration\nnot only priorit

PROMPT:
You are a helpful assistant whose job is to generate a JSON object.
You will be presented with a context and a set of output fields to generate. Your task is to generate a JSON object
which fills in the output fields with the correct values.
You will be provided with a description of each input field and each output field. All of the fields in the output 
JSON object can be derived using information from the context.

Remember, your answer must be a valid JSON list of dictionaries. The list may contain one or more dictionaries, and
each dictionary should only have the specified output fields. Finish your response with a newline character 
followed by ---
---
INPUT FIELDS:
- contents: The contents of the file
- filename: The UNIX-style name of the file
- text_contents: The text-only contents of the PDF

OUTPUT FIELDS:
- author_affiliation: The affiliation of the author.
- author_email: The email address of the author.
- author_name: The full name of the author.

CONTEXT:
{
  "contents": "<bytes>",
  "filename": "liu2024.pdf",
  "text_contents": "ARTICLE OPEN\nTowards precision oncology discovery: four less known genes\nand their unknown 
interactions as highest-performed\nbiomarkers for colorectal cancer\nYongjun Liu1, Yuqing Xu 2, Xiaoxing Li3 
\u2709, Mengke Chen3, Xueqin Wang 4, Ning Zhang5, Heping Zhang6 and\nZhengjun Zhang 2,7,8 \u2709\nThe goal of this 
study was to use a new interpretable machine-learning framework based on max-logistic competing risk factor\nmodels
to identify a parsimonious set of differentially expressed genes (DEGs) that play a pivotal role in the development
of\ncolorectal cancer (CRC). Transcriptome data from nine public datasets were analyzed, and a new Chinese cohort 
was collected to\nvalidate the\ufb01ndings. The study discovered a set of four critical DEGs - CXCL8, PSMC2, APP, 
and SLC20A1 - that exhibit the highest\naccuracy in detecting CRC in diverse populations and ethnicities. Notably, 
PSMC2 and CXCL8 appear to play a central role in CRC,\nand CXCL8 alone could potentially serve as an early-stage 
marker for CRC. This work represents a pioneering effort in applying the\nmax-logistic competing risk factor model 
to identify critical genes for human malignancies, and the interpretability and\nreproducibility of the results 
across diverse populations suggests that the four DEGs identi\ufb01ed can provide a comprehensive\ndescription of 
the transcriptomic features of CRC. The practical implications of this research include the potential for 
personalized\nrisk assessment and precision diagnosis and tailored treatment plans for patients.\nnpj Precision 
Oncology            (2024) 8:13 ; https://doi.org/10.1038/s41698-024-00512-1\nINTRODUCTION\nColorectal cancer (CRC)
is a signi\ufb01cant public health issue, being\none of the most prevalent human malignancies worldwide and\nthe 
second leading cause of cancer-related deaths1\u2013 3. While\nsurgical resection, chemoradiation, and 
immunotherapy have\nadvanced, they remain inadequate in many cases. Moreover, the\nincidence of CRC is increasing 
in younger individuals, particularly\nin the United States and other countries4\u2013 6. Genetic 
predisposition\nplays a crucial role in the development of CRC, with hereditary and\nsporadic causes accounting for
a signi\ufb01cant proportion of\ncases2,6,7. The etiology of CRC can be broadly classi\ufb01ed into 
two\ncategories: hereditary or sporadic. Hereditary CRC accounts for\n10\u221215% of the overall incidence and is 
attributable to mutations\nin APC or DNA mismatch repair genes. Sporadic CRC is more\nfrequent, representing >80% 
of CRCs, and is characterized by\nchromosomal instability, microsatellite instability (MSI), or CpG\nisland 
methylation6,7.\nOver the past decades, many transcriptomic studies have been\nperformed which have shed light on 
the molecular mechanisms\nunderlying CRC development, with a large number of genes being\nidenti\ufb01ed as 
differentially expressed bet

Total time: 10.14s
Total cost: $0.0000


[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
Exception in thread Thread-4:
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
Exception in threading.excepthook:

Traceback (most recent call last):
  File "/home/gerardo/.local/share/mamba/envs/bdf-pz/lib/python3.11/threading.py", line 1349, in invoke_excepthook
    hook(args)
  File "/home/gerardo/.local/share/mamba/envs/bdf-pz/lib/python3.11/site-packages/rich/file_proxy.py", line 43, in write
    with console:
  File "/home/gerardo/.local/share/mamba/envs/bdf-pz/lib/python3.11/site-packages/rich/console.py", line 864, in __exit__
    self._exit_buffer()
  File "/home/gerardo/.local/share/mamba/envs/bdf-pz/lib/python3.11/site-packages/rich/console.py", line 822, in _exit_buffer
    self._check_buffer()
  File "/home/gerardo/.local/share/mamba/envs/bdf-pz/lib/python3.11/site-packages/rich/console.py", line 2019, in _check_buffer
    self._write_buffer()
  File "/home/gerardo/.local/share/mamba/envs/bdf-pz/lib/python3.11/site-packages/rich/console.py", line 2035, in _write_buffer
    display(self._buffer, self._render_buffer(self._buffer[:]))
  File "/home/gerardo/.local/share/mamba/envs/bdf-pz/l

RecursionError: maximum recursion depth exceeded

In [ ]:
results_df

In [ ]:
print(stats.total_plan_time)